In [5]:
from graphsage.layers import GraphSAGELayer
from graphsage.utils import train
import torch_geometric.nn as geo_nn
import torch_geometric
import torch

In [3]:
# Fetch the Dataset object
dataset = torch_geometric.datasets.Planetoid(root='./data', name='Cora')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')
print()

data = dataset[0]
print(data)

Processing...
Done!
Number of features: 1433
Number of classes: 7

Data(edge_index=[2, 10556], test_mask=[2708], train_mask=[2708], val_mask=[2708], x=[2708, 1433], y=[2708])


# GCN baseline

Training a 2 layer GCN as baseline

In [14]:
torch.manual_seed(1)
model = geo_nn.Sequential('x, edge_index', [
    (geo_nn.SAGEConv(1433, 512), 'x, edge_index -> x'),
    torch.nn.ReLU(inplace=True),
    (geo_nn.SAGEConv(512, 7), 'x, edge_index -> x')
])

train(model, data, num_epochs=100, use_edge_index=True, learning_rate=0.001)

[Epoch 1/100] Loss: 1.9472153186798096 | Val: 0.116 | Test: 0.127
[Epoch 2/100] Loss: 1.792442798614502 | Val: 0.486 | Test: 0.498
[Epoch 3/100] Loss: 1.6447679996490479 | Val: 0.660 | Test: 0.662
[Epoch 4/100] Loss: 1.4941571950912476 | Val: 0.698 | Test: 0.709
[Epoch 5/100] Loss: 1.3365434408187866 | Val: 0.728 | Test: 0.731
[Epoch 7/100] Loss: 1.0059746503829956 | Val: 0.736 | Test: 0.743
[Epoch 8/100] Loss: 0.842917799949646 | Val: 0.744 | Test: 0.751
[Epoch 9/100] Loss: 0.6893827319145203 | Val: 0.750 | Test: 0.758
[Epoch 10/100] Loss: 0.5505876541137695 | Val: 0.752 | Test: 0.759
[Epoch 11/100] Loss: 0.430237352848053 | Val: 0.756 | Test: 0.765
[Epoch 14/100] Loss: 0.18587052822113037 | Val: 0.758 | Test: 0.773
[Epoch 15/100] Loss: 0.13779595494270325 | Val: 0.760 | Test: 0.773
[Epoch 17/100] Loss: 0.07549642771482468 | Val: 0.762 | Test: 0.784
[Epoch 23/100] Loss: 0.015097255818545818 | Val: 0.764 | Test: 0.786
[Epoch 100/100] Loss: 0.0004028693656437099 | Val: 0.766 | Test: 0.7

# Mean aggregator

First we train the mean pooling layers. With and without concatenation (using addition instead)

In [8]:
import torch_geometric.nn as geo_nn
torch.manual_seed(1)
model = geo_nn.Sequential('x, edge_index', [
    (GraphSAGELayer(1433, 7, "mean", depth=2, dropout=0.2, concat=True), 'x, edge_index -> x'),
])

train(model, data, num_epochs=100, use_edge_index=True, learning_rate=0.001)

[Epoch 1/100] Loss: 1.9460043907165527 | Val: 0.078 | Test: 0.128
[Epoch 2/100] Loss: 1.844359278678894 | Val: 0.582 | Test: 0.595
[Epoch 3/100] Loss: 1.7232178449630737 | Val: 0.662 | Test: 0.677
[Epoch 4/100] Loss: 1.5939891338348389 | Val: 0.668 | Test: 0.693
[Epoch 5/100] Loss: 1.448744773864746 | Val: 0.680 | Test: 0.696
[Epoch 6/100] Loss: 1.2858067750930786 | Val: 0.688 | Test: 0.703
[Epoch 7/100] Loss: 1.1064516305923462 | Val: 0.696 | Test: 0.714
[Epoch 8/100] Loss: 0.9226663708686829 | Val: 0.722 | Test: 0.738
[Epoch 9/100] Loss: 0.7443092465400696 | Val: 0.750 | Test: 0.767
[Epoch 10/100] Loss: 0.5802332162857056 | Val: 0.770 | Test: 0.783
[Epoch 22/100] Loss: 0.008163705468177795 | Val: 0.774 | Test: 0.778
[Epoch 28/100] Loss: 0.0017871992895379663 | Val: 0.776 | Test: 0.778


In [9]:
import torch_geometric.nn as geo_nn
torch.manual_seed(1)
model = geo_nn.Sequential('x, edge_index', [
    (GraphSAGELayer(1433, 7, "mean", depth=2, dropout=0.2, concat=False), 'x, edge_index -> x'),
])

train(model, data, num_epochs=100, use_edge_index=True, learning_rate=0.001)

[Epoch 1/100] Loss: 1.9460296630859375 | Val: 0.132 | Test: 0.142
[Epoch 2/100] Loss: 1.7889565229415894 | Val: 0.430 | Test: 0.456
[Epoch 3/100] Loss: 1.609548807144165 | Val: 0.590 | Test: 0.609
[Epoch 4/100] Loss: 1.4181588888168335 | Val: 0.692 | Test: 0.723
[Epoch 5/100] Loss: 1.2192357778549194 | Val: 0.750 | Test: 0.772
[Epoch 6/100] Loss: 1.0180740356445312 | Val: 0.762 | Test: 0.793
[Epoch 7/100] Loss: 0.8214982748031616 | Val: 0.776 | Test: 0.800
[Epoch 8/100] Loss: 0.6389662623405457 | Val: 0.784 | Test: 0.798


# Max learnable aggregator

We do the same with the max learnable pooling operator

In [10]:
import torch_geometric.nn as geo_nn
torch.manual_seed(1)
model = geo_nn.Sequential('x, edge_index', [
    (GraphSAGELayer(1433, 7, "max", depth=2, dropout=0.2, concat=True), 'x, edge_index -> x'),
])

train(model, data, num_epochs=100, use_edge_index=True, learning_rate=0.001)

[Epoch 1/100] Loss: 1.9446347951889038 | Val: 0.080 | Test: 0.104
[Epoch 2/100] Loss: 1.8690122365951538 | Val: 0.342 | Test: 0.329
[Epoch 3/100] Loss: 1.7360353469848633 | Val: 0.364 | Test: 0.379
[Epoch 4/100] Loss: 1.5317305326461792 | Val: 0.560 | Test: 0.574
[Epoch 5/100] Loss: 1.2590444087982178 | Val: 0.564 | Test: 0.591
[Epoch 6/100] Loss: 0.9614983797073364 | Val: 0.590 | Test: 0.606
[Epoch 7/100] Loss: 0.6655979752540588 | Val: 0.754 | Test: 0.752


In [12]:
import torch_geometric.nn as geo_nn
torch.manual_seed(1)
model = geo_nn.Sequential('x, edge_index', [
    (GraphSAGELayer(1433, 7, "max", depth=2, dropout=0.2, concat=False), 'x, edge_index -> x'),
])

train(model, data, num_epochs=100, use_edge_index=True, learning_rate=0.001)

[Epoch 1/100] Loss: 1.9499422311782837 | Val: 0.126 | Test: 0.142
[Epoch 2/100] Loss: 1.8980891704559326 | Val: 0.156 | Test: 0.144
[Epoch 3/100] Loss: 1.7474186420440674 | Val: 0.322 | Test: 0.372
[Epoch 4/100] Loss: 1.608534336090088 | Val: 0.426 | Test: 0.471
[Epoch 5/100] Loss: 1.394709825515747 | Val: 0.546 | Test: 0.572
[Epoch 6/100] Loss: 1.1636906862258911 | Val: 0.612 | Test: 0.602
[Epoch 9/100] Loss: 0.7053164839744568 | Val: 0.654 | Test: 0.656
[Epoch 11/100] Loss: 0.6020401120185852 | Val: 0.660 | Test: 0.676
[Epoch 14/100] Loss: 0.5652309656143188 | Val: 0.662 | Test: 0.688
[Epoch 15/100] Loss: 0.5605665445327759 | Val: 0.674 | Test: 0.701
[Epoch 16/100] Loss: 0.5583226680755615 | Val: 0.686 | Test: 0.711
